# 隠れマルコフモデル使って系列解析してみる


## 必要なライブラリのimport
`nltk`
- 簡単に使えそうなコーパスだったので採用
- `pip install nltk`でインストールできる（venvにはすでにインストール済み）

`nltk`のサイズは大きいので, 必要な二つのデータだけ持ってきます
- `treebank`
    - 大規模注釈付きコーパス
- `universal_tagset`
    - 品詞タグのセット


In [1]:
import nltk
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/shibatakeigo/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/shibatakeigo/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

## 条件付き確率の計算
HMMを作成するには
1. $ P(\bm{z_t} | \bm{z_{t-1}}) $ 
2. $ P(\bm{x_t} | \bm{z_{t}}) $
を用意すればOK.

例えば, 今回使う文章（教科書からのパクリ）「Times flies like an arrow」だと
1. $ P(V | V) $
    V (動詞)の後にV（動詞）が出る確率
2. $ P(V | \rm{like}) $
    likeという単語がV（動詞）である確率

### 実装方針
1. 単語とタグのペアを取得する
- words_tags
- 全部小文字にする（簡単化）
2. 頻度分布を作成
- 何回の頻度でその単語が出たのか？
- frq_dis\[word\]\[label\]に`word`の品詞が`label`であった回数が格納される

3. 条件付き確率分布を求める
- $ P(品詞 | 単語) $で「単語」が出た時の「品詞」の確率を表す


In [2]:
from nltk.corpus import brown

# Get pare of words and tags
words = brown.words()
words_tags = [(word.lower(), tag) for word, tag in brown.tagged_words(tagset='universal')]

# print example
print(words_tags[:10])

[('the', 'DET'), ('fulton', 'NOUN'), ('county', 'NOUN'), ('grand', 'ADJ'), ('jury', 'NOUN'), ('said', 'VERB'), ('friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP')]


In [3]:
# 頻度分布を作成

word_list = list(set([word_tag[0] for word_tag in words_tags]))
tag_list = list(set([word_tag[1] for word_tag in words_tags] + ["BOS"])) # BOSは文の先頭を表すタグ

freq_dist = {
    word: {
        tag: 0 for tag in tag_list
    } for word in word_list
}

for word, tag in words_tags:
    freq_dist[word][tag] += 1
        
print(freq_dist["times"])


{'NOUN': 296, 'BOS': 0, 'ADP': 3, '.': 0, 'ADV': 0, 'PRON': 0, 'X': 0, 'NUM': 0, 'CONJ': 0, 'DET': 0, 'PRT': 0, 'VERB': 1, 'ADJ': 0}


In [4]:
# 文章に分割する
# words = brown.words()
sentences = []
sentence = []
for word_to_sentence in words_tags:
    if word_to_sentence[0] != '.':
        sentence.append(word_to_sentence)
    else:
        sentence.append(word_to_sentence)
        sentences.append(sentence)
        sentence = []

In [5]:
# # 条件付き確率分布を作成

# それぞれのタグの出現回数を計算
C_word = {
    tag: 0 for tag in tag_list
}

for word_tag in words_tags:
    C_word[word_tag[1]] += 1

# BOSの出現回数を追加
C_word["BOS"] = len(sentences)

# 条件付き確率を計算
cfd = {
    tag: {
        word: freq_dist[word][tag] / C_word[tag] if C_word[tag] != 0 else 0 for word in word_list
    } for tag in C_word.keys()
}

sum(cfd["ADP"].values())


0.9999999999999997

In [6]:
print(len(sentences)) #文章の数を出力
print(sentences[0]) #最初の文章を出力

# 品詞のbigramを作成
bigram = []
for sentence in sentences:
    for index in range(len(sentence)):
        if index == 0:
            bigram.append(["BOS", sentence[index][1]])
        else:
            bigram.append([sentence[index - 1][1], sentence[index][1]])

print(bigram[0]) #最初の文章のbigramを出力

49346
[('the', 'DET'), ('fulton', 'NOUN'), ('county', 'NOUN'), ('grand', 'ADJ'), ('jury', 'NOUN'), ('said', 'VERB'), ('friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]
['BOS', 'DET']


In [7]:
# C(品詞, 品詞)を計算
tag_bigram = {
    tag1: {
        tag2: 0 for tag2 in tag_list
    } for tag1 in tag_list
}

for word_tag1, word_tag2 in bigram:
    tag_bigram[word_tag1][word_tag2] += 1
    
# 品詞間（隠れ状態）の遷移確率を計算 P(品詞 | 品詞)
transition_pro = {
    tag1:{
        tag2: tag_bigram[tag1][tag2] / C_word[tag1] if C_word[tag1] != 0 else 0 for tag2 in tag_list
    } for tag1 in tag_list
}

# C(ADP | ADP)
print(tag_bigram["ADP"]["ADP"])
# P(ADP | ADP)
print(transition_pro["ADP"]["ADP"])
print(transition_pro["BOS"])
print(C_word["BOS"])

2941
0.02031554370501361
{'NOUN': 0.13861305880922467, 'BOS': 0.0, 'ADP': 0.12766992258744375, '.': 0.08235723260243992, 'ADV': 0.09477971872086896, 'PRON': 0.16337697077777327, 'X': 0.0005268917440116727, 'NUM': 0.017184776881611477, 'CONJ': 0.043063267539415556, 'DET': 0.22054472500303976, 'PRT': 0.03704454261743606, 'VERB': 0.040307218416892956, 'ADJ': 0.034531674299841934}
49346


## Viterbiアルゴリズム
与えられた観測系列（ここでは単語）を元に, 隠れ状態（ここでは品詞）の最も確からしい系列を求めたい.
動的計画法（DP）で実装することができる

### DPテーブルの初期化
DPテーブルの配列サイズは $ K \times N$になる. ここで, $ K $は状態数（ここでは品詞の数）$ N $（ここでは観測した系列の長さ, つまり文字列の長さ）になる 
まず, 先頭の隠れ状態の確率分布 $ P(\bm{z}_0) $ を決定する必要があるが, 品詞付与タスクの場合は明らかに $ P(\rm{"BOS"}) = 1 $でその他の品詞が出る確率は0. （"BOS"は先頭の特殊文字）
DPテーブルの初期化式は
$$
    V[0][k] =
        \left\{
            \begin{array}{ll}
                1 & k = \rm{"BOS"} \\
                0 & \rm{その他の品詞}
            \end{array}
        \right.

$$

### DPテーブルの更新
`V[i][k]`（入力された文字のi番目の品詞がkである確率の最大値（尤度?））は`V[i - 1]`を使って次のように更新できる
$$
    V[i][k] = 
        \rm{max}\left\{
            \begin{array}{ll}
                V[i - 1][\rm{"BOS"}] \\
                V[i - 1][\rm{"ADP"}] \\
                V[i - 1][\rm{"VERB"}] \\
                \cdots \\
            \end{array}
        \right.
$$

### 計算量
状態数を`K`, 文字列の長さを`N`とした時, `O(NK)`

### 実験
三つの文字列で試してみる
- `Time flies like an arrow"`
    - 光陰矢の如し
    - 時バエは矢を好む
- `Book covers like a jacket`
    - 本はジャケットのようにカバーしている
    - ブックカバーはジャケットのようだ

このデータセットが小さいからなのかわからないけど, `like`が前置詞として解釈される
あとは正常？かな

### 品詞一覧
- ADP : 前置詞
- CONJ : 接続詞
- ADJ : 形容詞
- . : 句読点
- VERB : 動詞
- ADV : 副詞
- NUM : 数詞
- BOS : 先頭文字（勝手に作った特殊記号）
- PRT : 助詞
- NOUN : 名詞
- DET : 限定詞(theとかanとか)
- PRON : 代名詞
- X : その他もろもろ

In [9]:
V = [{}]
path = {}
start_p = {
    tag: 1 if tag == "BOS" else 0 for tag in tag_list
}
# sample_str = "Time flies like an arrow"
# sample_str = "he likes an apple"
sample_str = "Book covers like a jacket"
words_list = list(map(lambda s: s.lower() ,sample_str.split()))
# # 初期化
for tag in tag_list:
    if tag == "BOS":
        V[0][tag] = 1
    else:
        V[0][tag] = 0
    path[tag] = [tag]
    
for index in range(len(words_list)):
    V.append({})
    newpath = {}
    
    for cur_tag_state in tag_list:
        prob, state = max((V[index][y] * transition_pro[y][cur_tag_state] * cfd[cur_tag_state][words_list[index]], y) for y in tag_list)
        V[index + 1][cur_tag_state] = prob
        newpath[cur_tag_state] = path[state] + [cur_tag_state]
    
    path = newpath
    
(prob, state) = max((V[len(words_list) ][y], y) for y in tag_list)

print(prob)
print(path[state])

9.322601105263117e-18
NOUN
['BOS', 'NOUN', 'NOUN', 'ADP', 'DET', 'NOUN']
